In [5]:
#from here
import numpy as np
import tensorflow as tf
from pathlib import Path
from PIL import Image
import pandas as pd

# === Load TFLite Model ===
tflite_model_path = "mobilenet_distilled_v2.tflite"
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# === Get input shape ===
input_shape = input_details[0]['shape']  # Should be [1, height, width, 3]
input_height = input_shape[1]
input_width = input_shape[2]
print(f"Expected input shape: {input_shape}")


Expected input shape: [  1   3 224 224]


In [6]:
# === Image Preprocessing ===
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = image.resize((input_width, input_height))
    image_np = np.array(image, dtype=np.float32)

    # Normalize if required (e.g., MobileNet normalization)
    image_np = image_np / 255.0

    # Reshape to match input
    image_np = np.expand_dims(image_np, axis=0)  # shape: (1, H, W, 3)
    return image_np

In [7]:
# === Prediction Function ===
def predict_image(image_path, threshold=0.85):
    input_data = preprocess_image(image_path)

    # Set the tensor
    interpreter.set_tensor(input_details[0]['index'], input_data)

    # Run inference
    interpreter.invoke()

    # Get output
    output_data = interpreter.get_tensor(output_details[0]['index'])  # shape: (1, N)
    probabilities = output_data[0]

    pred_idx = np.argmax(probabilities)
    pred_prob = probabilities[pred_idx]

    flag = 1 if pred_prob >= threshold else 0

    return flag, float(pred_prob), int(pred_idx)

In [8]:
# === Image Folder ===
base_path = Path(r"C:\Users\arpitasharma2\OneDrive - Deloitte (O365D)\Desktop\DIU\NMMS PHOTOS OF 10TH APRIL\New folder")
image_paths = list(base_path.glob('*.jpeg')) + list(base_path.glob('*.jpg')) + list(base_path.glob('*.png'))

print(f"Found {len(image_paths)} images. Starting inference...\n")

# === Run Predictions ===
results = []

for img_path in image_paths:
    try:
        flag, prob, pred_class = predict_image(img_path)
        results.append({
            'Image': str(img_path),
            'Class': pred_class,
            'Flag': flag,
            'Probability': prob
        })
    except Exception as e:
        print(f"Error with image {img_path.name}: {e}")

# === Save Results ===
df_results = pd.DataFrame(results)
df_results.to_csv("tflite_predictions.csv", index=False)
print("\nSample results:")
print(df_results.head())

Found 17 images. Starting inference...

Error with image 10_1708005082_RC_22012034837192_2025-04-10R1042025111233.jpeg: Cannot set tensor: Dimension mismatch. Got 3 but expected 224 for dimension 3 of input 0.
Error with image 10_1710004041_WC_22012035247470_2025-04-10R1042025023342.jpeg: Cannot set tensor: Dimension mismatch. Got 3 but expected 224 for dimension 3 of input 0.
Error with image 10_1722_WC_22012035192133_2025-04-10R1042025101043.jpeg: Cannot set tensor: Dimension mismatch. Got 3 but expected 224 for dimension 3 of input 0.
Error with image 11_1710004041_WC_22012035247470_2025-04-10R1042025023353.jpeg: Cannot set tensor: Dimension mismatch. Got 3 but expected 224 for dimension 3 of input 0.
Error with image 12_1715007030_WC_22012035242521_2025-04-10R1042025111803.jpeg: Cannot set tensor: Dimension mismatch. Got 3 but expected 224 for dimension 3 of input 0.
Error with image 12_1722_WC_22012035192133_2025-04-10R1042025101106.jpeg: Cannot set tensor: Dimension mismatch. Got